In [1]:
import pickle
from pprint import pprint
import psycopg2

In [12]:
dag_ids = ['stock']
READ_COST_TEMPLATE = "./include/dag_timings/{}_timing_logs.pkl"
TOTAL_COST_TEMPLATE = "./include/dag_timings/{}_task_durations.pkl"

read_costs = {}
total_costs = {}
for dag_id in dag_ids:
    read_cost_file = READ_COST_TEMPLATE.format(dag_id)
    total_cost_file = TOTAL_COST_TEMPLATE.format(dag_id)
    with open(read_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        read_costs[dag_id] = loaded_data
    with open(total_cost_file, "rb") as f:
        loaded_data = pickle.load(f)
        total_costs[dag_id] = loaded_data

print(f"Files: {read_cost_file}, {total_cost_file}")
print('READ COSTS:')
pprint(read_costs)
print('TOTAL COSTS:')
pprint(total_costs)


Files: ./include/dag_timings/example_timing_logs.pkl, ./include/dag_timings/example_task_durations.pkl
READ COSTS:
{'distant_example_high_read_cost': {'task5': {('xcom', 'amount'): 0.003681659698486328}},
 'example': {'dispense20': {('xcom', 'amount', 'read'): 0.0034945011138916016}}}
TOTAL COSTS:
{'distant_example_high_read_cost': {'task1': 1.3530879999999998,
                                    'task2': 1.2573460000000003,
                                    'task3': 1.3690200000000001,
                                    'task4': 1.046855,
                                    'task5': 1.174066,
                                    'task6': 1.2911190000000001},
 'example': {'dispense20': 0.358372, 'dispense50': 0.460421}}


In [8]:
dag_id = 'example'
POSTGRES_CONFIG = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': 'localhost',  # will be updated dynamically to point to container IP
    'port': 5432,
}
conn = psycopg2.connect(**POSTGRES_CONFIG)

query = """
        SELECT *
        FROM task_instance
        WHERE dag_id = %s
        LIMIT 1;
    """
with conn.cursor() as cur:
    cur.execute(query, (dag_id,))
    results = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]

conn.close()
results

OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
